In [132]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
data = pd.read_csv('HR-Employee-Attrition.csv')

data_classification = data[['Attrition']]
data_values = data.loc[:, data.columns != "Attrition"]
data_train, data_test, labels_train, labels_test = train_test_split(data_values, data_classification, test_size=0.2, random_state=35)
# print(f"{data_train.shape} {data_test.shape}, {labels_train.shape}, {labels_test.shape}")

In [134]:
# Data Cleaning

#1 Convert text data to numerical

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder() # since all the non-numerical features are categorical, it should be safe to use label encoding

labels_train['Attrition'] = encoder.fit_transform(labels_train['Attrition'])
labels_test['Attrition'] = encoder.transform(labels_test['Attrition']) # don't fit encoder with testing labels, but transform testing labels with same fitted encoder

features_to_encode = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']
for feature in features_to_encode:
#     data[feature] = encoder.fit_transform(data[feature])
    data_train[feature] = encoder.fit_transform(data_train[feature])
    data_test[feature] = encoder.transform(data_test[feature]) # don't fit encoder with testing data, but transform testing data with same fitted encoder

In [139]:
#2 Feature Selection

from sklearn.feature_selection import SelectKBest, chi2

feature_selector = SelectKBest(chi2, k=30) # drop the 4 features that have the lowest correlation with the answer
data_train_selected = feature_selector.fit_transform(data_train, labels_train)
data_test_selected = feature_selector.transform(data_test)

(1176, 34) (1176, 30)
(1176, 34) (1176, 30)


In [142]:
#3 Normalize

from sklearn.preprocessing import StandardScaler

normalize_scaler = StandardScaler() 
# data_normalized = normalize_scaler.fit_transform(data_values_selected, np.ravel(data_classification))
data_train_clean = normalize_scaler.fit_transform(data_train_selected, labels_train)
data_test_clean = normalize_scaler.transform(data_test_selected)

# use in model fitting: data_train_clean, data_test_clean, labels_train, labels_test


In [ ]:
#4 Extra Credit?: graphing

In [13]:
# Model Training

In [14]:
#1 Split the data (stratify too)

In [15]:
#2 K Nearnest Neighbors

In [16]:
#3 Decision Tree

In [17]:
#4 Linear Regression

In [18]:
#5 Support Vector Machines

In [19]:
#6 Artificial Neural Networks

In [ ]:
#TODO: 
    # comment out the normalize section and see how the neural network score changes.
    # don't want to do cleaning like feaure selection and normalization on the data before splitting it into testing and training.
        # instead, put all the cleaning into a function and pass the split training and test data into it so that each one is 
        # cleaned individually instead of together.  Put a comment to note that you're doing this
        #all you need is the split data, then pass it into the function so that it returns numerical, feature-selected, 
        # HOWEVER, use the same StandardScaler() and SelectKBest() object (maybe even the same LabelEncoder()) as stated by https://www.edwinwenink.xyz/posts/61-feature_preprocessing_and_train_test_leakage/
    # remove commented code
        